## Week 3 of the Capstone Project for the IBM Professional Data Science Certificate - Coursera

##### Import necessary libraries

In [44]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
print ('Pandas,Numpy & BeatufiulSoup have been imported!')

Pandas,Numpy & BeatufiulSoup have been imported!


Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [36]:
sourceweb = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(sourceweb, 'html.parser')

##### Getting the required information from the website without any further editting.

In [37]:
postalCodeList = []
boroughList = []
neighborhoodList = []

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

##### Fill the table with the list from the website - 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [38]:
#Fill  dataframe with list
toronto_neighorhood = [('PostalCode', postalCodeList),
                      ('Borough', boroughList),
                      ('Neighborhood', neighborhoodList)]
toronto_df = pd.DataFrame.from_dict(dict(toronto_neighorhood))
toronto_df.head()

,Borough,Neighborhood,PostalCode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


##### Remove cells with "Not assigned" under borough and grouping neighborhoods sharing the same postal code

In [39]:
toronto_df_dropna = toronto_df[toronto_df.Borough != 'Not assigned'].reset_index(drop=True)
toronto_df_grouped = toronto_df_dropna.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


##### Making unassigned neighborhoods the same as their respective borough. 

In [40]:
na_neigh_rows = toronto_df_grouped.Neighborhood == 'Not assigned'
toronto_df_grouped.loc[na_neigh_rows, 'Neighborhood'] = toronto_df_grouped.loc[na_neigh_rows, 'Borough']

##### Printing the number of rows of the dataframe after cleaning the data.

In [41]:
toronto_df_final = toronto_df_grouped
toronto_df_final.shape

(103, 3)

## Part 2

##### Getting latitude and longitude from the website specified as source

In [50]:
source2 = "http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
coor = pd.read_csv(source2)
coor.columns = ['PostalCode', 'Latitude', 'Longitude']
coor.rename({'Postal Code':'PostalCode'},axis=1, inplace=True)
coor.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### Creating a new dataframe with longitude and latitude  by merging the postal code, borough and neigborhood according to each respective postal code

In [49]:
df_LALO = pd.merge(toronto_df_final,coor,how='left', on='PostalCode')
df_LALO.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
